In [13]:
import pandas as pd
import re

df= pd.read_excel('2019 google BAD sample(1-12).xlsx')
df.columns =['ID','label','C1']

In [22]:
!pip install google_trans_new

In [24]:
from google_trans_new import google_translator  
import googletrans
from googletrans import Translator
 
translator = Translator()
value = translator.translate("Good morning", src='en', dest='ko')
 
print(value)
print(value.src)  # 변환할 언어
print(value.dest)  # 변환될 언어
print(value.text)  # 변환 결과 




AttributeError: 'NoneType' object has no attribute 'group'

In [15]:
#fasttext에서 사용될 포맷으로 저장하려함
train = open('google_train','w',encoding='utf-8')
   

In [16]:
def stopwords(text):
    stopwords = "test 감사 안함 모름 모르 무료 없음 기억안남 없다 기억 몰라 무슨"
    stopwords_arr = stopwords.split(' ')
    for sw in stopwords_arr:
        if len(re.compile(sw).findall(text)) > 0:
            return True
    
    return False

In [17]:
#fasttext 포맷으로 저장
for index,row in df.iterrows():
    if stopwords(str(row["C1"])) == False :
        print(f'__label__{row["label"]} {row["C1"]}', file=train)

# Fasttext

In [18]:
import fasttext
model = fasttext.train_supervised('google_train')
#model.save_model("google_train.bin")
#model = fasttext.load_model('google_train.bin')


In [19]:
print(model.labels)

['__label__0', '__label__1']


In [20]:
#model.test("google_train", k=5)
model.test("google_train")
#모델 건수,정밀도,재현율

#model.predict("xx")


(27221, 0.9936446126152603, 0.9936446126152603)

In [21]:
dfTest= pd.read_excel('2019 google BAD sample(1-12).xlsx')
dfTest.columns =['ID','label','C1']
dfTest['label_predict'] = ''
dfTest['acc'] = ''


In [9]:
import re
text = "ksjakdlf 카카오뱅크"
textArr = re.compile('[a-zA-z가-핳]').findall(text)
enLen = len(re.compile('[a-zA-z]').findall(text))
korLen = len(re.compile('[가-핳]').findall(text))
text = ''.join(textArr)
text


'ksjakdlf카카오뱅크'

In [22]:
def cleanText(text):
     #텍스트에 포함되어 있는 특수 문자 제거
    text = text.replace('\\','')
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text

In [ ]:
def searchDuplicate(text,position):
    stopwords = "test 감사 안함 모름 모르 무료 없음 기억안남 없다 기억 몰라 무슨"
    stopwords_arr = stopwords.split(' ')
    '''
    1. 영어/한글 분리작업 (ㅋㅋ ㅎㅎ 제거 )
        - 영어인 경우 3자 이하 false
        - 중복패턴 체크(2자)
        - 한글 경우 1자 이하 false

    2. 숫자만으로 이루어진 경우 false
    3. 번역이 되지 않는다면 False
    4. 제거 리스트 test, 감사, 안함, 모름, 모르 , 무료
    5. 중복 발견시 false
    6. fasttext
    7. 이 다음이 영어 /한국어 형태소 분석 <-- 의미 없음
    '''
    text = cleanText(text)
    #영어, 한글만, 
    
    textArr = re.compile('[a-zA-z가-핳]').findall(text)
    enLen = len(re.compile('[a-zA-z]').findall(text))
    korLen = len(re.compile('[가-핳]').findall(text))
    text = ''.join(textArr)
    
    languageFlag = 0
    if len(textArr) == enLen: #영어
        languageFlag = 1    
        #position =3
    elif len(textArr) == korLen:  #한국어
        languageFlag = 2    
    else:
        languageFlag =3 #혼용

    #text없다면 False
    if text == '':
        return False
        
    #영어 번역기를 통해
    if languageFlag == 1:
        if text.lower().strip() == translator.translate(text.lower(),lang_tgt='ko').strip()  :
            #print('번역X :' + text)
            return False
        #else : 
            #print('번역가능:' + text)
    
    #자리수 부족 False 여기 수정
    if (languageFlag == 1 and enLen<=3) or (languageFlag==2 and korLen<=1):
        return False
    
    
    
    #stopwords  포함되면  out
    for sw in stopwords_arr:
        if len(re.compile(sw).findall(text)) > 0:
            return False
    x = len(text)
    if x >= position:
        for i in range(0, x-position+1):
            dupStrArray = re.compile(text[i:i+position]).findall(text)
            #print(text[i:i+position]+':'+str(len(dupStrArray)))
            if len(dupStrArray)>1: #중복발생시  out
                return False
        return True
    else:
        return False
    
   

In [ ]:
for index,row in dfTest.iterrows():
    if searchDuplicate(str(row["C1"]),2) :
        predictArr = model.predict(str(row["C1"]))
        dfTest.at[index,"label_predict"] = int(predictArr[0][0].replace('__label__',''))
        dfTest.at[index,"acc"] = str(predictArr[1][0])
    else :
        dfTest.at[index,"label_predict"] = 1
        dfTest.at[index,"acc"] = ''
    
    

In [ ]:
dfTest

In [13]:
dfTest.to_excel('google_result.xlsx',sheet_name='result')
with pd.ExcelWriter('google_result.xlsx', mode='a') as writer:
    df.to_excel(writer, sheet_name='source')